In [97]:
from bs4 import BeautifulSoup as bs
import csv
import datetime
import re
import requests

This code works because the San Francisco government uses Granicus to host all of their commission minutes. There's a consistent format for each page!

In [82]:
commissions = [
    ('Planning Commission', 'http://sanfrancisco.granicus.com/ViewPublisher.php?view_id=20'),
    ('Rules Committee', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=13'),
    ('Land Use & Transportation', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=177'),
    ('Government Audit & Oversight', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=11'),
    ('Budget & Finance', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=7'),
    ('Budget & Finance', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=189'),
    ('Board of Supervisors', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=10'),
    ('Board of Appeals', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=6'),
    ('Building Inspection', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=14'),
    ('Community Investment and Infrastructure', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=169'),
    ('Transportation Authority', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=24'),
    ('Historic Preservation', 'https://sanfrancisco.granicus.com/ViewPublisher.php?view_id=166'),
]

In [68]:
def get_table(soup):
    """
    Parameters:
    soup (BeautifulSoup object): The HTML of the entire commission
        minutes website.
    
    Returns:
    BeautifulSoup object: The HTML containing the table listing
        meeting dates and minutes.
    """
    tables = soup.findAll('table')

    table = None
    for t in tables:
        if 'Caption Notes' in t.get_text():
            table = t
            break
    return table

In [84]:
def get_all_minutes_links(table):
    """
    Parameters:
    table (BeautifulSoup object): The HTML containing the table
        listing meeting dates and minutes.
    
    Returns:
    [(string, string)]: A list of tuples containing the meeting date
        stored as a string and a link to the meetings' minutes.
    """
    minutes_links = []

    for td in table.findAll('td'):
        if 'Date' in td['headers']:
            if td.span != None:
                td.span.extract()
            date_str = td.get_text()

            date = datetime.datetime.strptime(date_str, '%m/%d/%y')
            if date < datetime.datetime(2019, 1, 1, 0, 0):
                break

        if 'Minutes' in td['headers']:
            a_tag = td.find('a')
            minutes_links.append((date_str, 'http:%s' % a_tag['href']))
    return minutes_links

In [85]:
# Get a list of the dates, minute links, and commission names for
# all meetings of every commission in our input list.

minutes_list = []
for name, link in commissions:
    r = requests.get(link)
    soup = bs(r.content)
    table = get_table(soup)
    dates_and_links = get_all_minutes_links(table)
    minutes_list += [(date, link, name) for date, link in dates_and_links]

[('12/11/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34700', 'Government Audit & Oversight'), ('12/05/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34656', 'Government Audit & Oversight'), ('11/21/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34577', 'Government Audit & Oversight'), ('11/13/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34490', 'Government Audit & Oversight'), ('11/07/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34457', 'Government Audit & Oversight'), ('10/23/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34357', 'Government Audit & Oversight'), ('10/17/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34299', 'Government Audit & Oversight'), ('10/03/19', 'http://sanfrancisco.granicus.com/TranscriptViewer.php?view_id=11&clip_id=34167', 'Governm

In [95]:
# Generate a list of dictionaries representing CSV rows. Each
# comment made that contains the substring 'trump' will have
# its own row in the output.

# Each row contains the date of the meeting, a link to the
# meeting minutes, commission name, and full text of the comment
# that contains the substring 'trump'.

# Known error: some of the rows ge

rows = []
for date, link, commission in minutes_list:
    r = requests.get(link)
    soup = bs(r.content)
    text = soup.get_text()
    
    index = text.lower().find('public comment')
    if index > 0:
        text = text[index:]
    
    if 'trump' in text.lower():
        indices = [m.start() for m in re.finditer('trump', text.lower())]
        for index in indices:
            before = text[:index]
            comment_start_index = before.rfind('>>')

            after = text[index:]
            comment_end_index = after.find('>>') + index

            comment = ''
            if comment_start_index != -1 and comment_end_index != -1:
                comment = text[comment_start_index:comment_end_index].replace('\n', ' ')
                print(comment + '\n')

            rows.append({"Date": date, "Link to Minutes": link, "Commission": commission, "Comment": comment})

>> good morning and thank you  supervisor and thank you supervisors.  As mentioned, our office has been working with the coverage  apartments to initiate this policy and those departments  include the department of public  health, department of homelessness and community development, department of children and youth and families and department of human services and so you'll hear from them  today.  As supervisor mandleman  mentioned, there was a  state-wide ordinance ab959, lbgt reduction's act championed in  2019 and an additional policy in  2017 and locally in 2016 through chapter 104 of the  administrative code.  As we look at data collection  for lgbt folks in the census,  the soji data becomes much more important, because we're documents despairties, as well as identifying areas and improvement across current services throughout the city,  which champions itself on  inclusion for lgbt the community, as billion as  as well as to  identify and address policy gaps and this underscore

>> I would respectfully request  a three minute interval.  Not only am I a black man, not only is it black history month,  not only did they give us the  shortest month, right now, I'm talking about african  americans, from the good black  dirt, we are in a state of emergency. Now if nobody in the city, the  supervisor, the mayor, don't want to recognize that, I'm going to identify it and  amplify it. Now, happy black history month, everyone. Excuse me.  I'm a little emotional.  I didn't take my high blood pressure medication today.  But I'm a great grandfather already.  I'm -- like the bible say, I'm  part of the three c's, my children's children's children.  any of our elected officials who don't have children, you cannot bear the responsibility  that a parent has to go to.  Dammit, in ten years, we are  going to be obsolete like the  indians, but I'm going to go up  to sacramento and knock on newsom newsom's door. And then, I'm going to washington and telling trump what we done lost

>> so I want to begin with -- I  was stalking you last night on  twitter -- and don't worry, you won't even know I'm there.  And I noticed a tweet from you that I just want to get a little bit of a read about  where we are right now in this moment because this has been  a -- a troubling week for  democracy in this country, and  you tweeted a statement by the federal elections commission  ellen weintraub expressing her  dismay and distress, and the fact that she would not allow  her office to endorse or abet a  foreign power providing opposition information to a candidate in an election, which donald trump said he would accept.  I'm just wondering about that, what do you think about this  moment and what do we all need  to do to ensure that our elections are fair and free from corruption?  

>> hello, ladies. [Applause] Are you having a good time  today?  [Cheers] Now we have our special guest here joining us.  Our leader, our speaker, the  woman who basically is the only  woman with co

>> good afternoon. Members of the committee my name  is hung and I represent chinese for action. I want to thank you for your wonderful work. This budget season to make sure we're expanding access and ensuring transparency in this year's budget process.  You and your offices have heard  from the immigrant voting collaborative. We're excited and look forward  to the proposed spending plan coming out today.  We look forward that this spending plan would direct some of these cost savings that have  been identified earlier to go  towards community initiatives that would benefit marginalized communities in san francisco  says it pertains to budget request.  We are very -- we're very a tune to the need that exist in san francisco through strategic partnerships with the san francisco unified school direct, the department of elections and our community-based  organizations in the collaborative.  We've been able to serve 60,000  san franciscos in the five-month ramp up towards the last election

>> earlier today, house speaker  nancy pelosi introduced  articles of impeachment to  impeach President Donald john trump. Donald trump solicited the interference of a foreign government to influence the  2020 united states presidential election and in so doing, used  the powers of the presidency in  a manner that compromised the national security of the united states and undermined the integrity of the united states democratic process.  His actions have undermined our  security and threatened our  2020 elections and violated his oath of office. As elected officials, each of  us took an oath to uphold in true faith the constitution of the united states of america. The constitution provides the house of representatives the sole power of impeachment and provides that the president  shall be removed from office on impeachment for and conviction  of treason, bribery or other high crimes and misdemeanors. San francisco has led the  resistance to donald trump's policies. Now the time has com

...

>> Supervisor Peskin:   I think  this is the first time since  1976 that san francisco has had two representatives on the coastal commission.  We had a -- I really got to  know her when we held a reception for her at the san francisco yacht club. She hails from supervisor haney's district, and she used law, policy, and communications  to defend californiians' rights to a healthy environment. I actually know her to be more  of a fierce warrior for environmental justice and  someone who has brought some much appreciated humor to the california coastal commission where we meet around the state once a month.  She never shies away from  peppering presenters or should  I say unknowing victims with  lines of questioning.  Her student note, 2006, on the  human rights impacts of sea  level rise, was one of the first legal papers to recognize  the impacts of climate change  on underserved and disadvantages communities not only in california but around the world.  She went onto author a chapter  

>> I'm marvin harrell Jr. I'm with the housing authority.  In January of 2018, I filed a  complaint with the department of  fair employment and housing for  hiring practices and promotional practices at the san francisco housing authority. There are no african american  males in any upper management  positions. In February, I filed a complaint with the san francisco housing authority board, the commissioners, along the same lines. But also relating to their  financial practices as a result.  And as it relates to the rent  increases, and then in March, I  filed a complaint with hud inspector general's office with  the financial aspect as it  relates to rent increases and their hiring practices, etc.  in August 2018, the housing  authority placed me on ad ministrative leave for trumped up charges that have yet to be proven with any truth to it.  I'm on paid administrative leave for eight months. The relationship that I'm here today -- I could retire tomorrow -- the reason that I'm here t

>> in the afternoon.  My name is paul, and I just want to show that here in san  francisco, we have no shortage of exceptional political leaders today we have the tremendous privilege of selecting a board  President Among very qualified  pool of leaders in san francisco some of the best leaders that the city has to offer. I would like to add my voice and  during the course of community to support hilary ronen for board President See hilary is a  type of person who stops and  thinks to herself, what can I do today to serve those who are hurting most among us while  those who need most among us while she is a person who shows  up for us every single day. She shows up for homeless families, she shows up for immigrant communities, and she shows up for public school  families. We have two years left in the trump presidency. Right now we have the opportunity to elect a President  To a standard form and defend  our values as san franciscans. We are above the double standard  of awarding the a

>> hello.  My name the commissioner dennish richards. I don't enjoy standing up here. I didn't ever want to stand up  here to hear somebody our partnership hired had sloppy drawings pains me.  You ask why we are here. I'm being retaliated against.  The address is 3847 -- 3849 18th street.  We are hearing it on December 18. We have heard it twice before. You have the videos in your brief from us.  Please watch it.  That's why Mr. Hernandez and Mr.  Duffy were so tongue-tied around  what the hell was so wrong with the building that we had to revoke the permits. What safety issues were there? It's me. That's why they are doing this. They are trying to get back at me. They are holding it over my head  so that I let that project go  and stop pursuing it.  Commissioner honda called me up. Offered a quid pro quo,  basically, and I said to him, look, darryl, I told you when I  first met you, I cannot be bought. And I still cannot be bought. And I feel uncomfortable talking  about this right no

>> Miss Selby, thank you very much.  In the last couple of months, I have met with two companies that  claim to do electric cars for tn cs and they are applying to  the california puc for licenses, using electric vehicles. That is simply not acceptable.  So if you want to look into the  future, we are not -- so this is -- I think this is tricky is  I'll be frank with you, because  it is about this feeling, if you  use an electric car, we're lowering emissions and it's ok. It is still not ok because san  francisco has invested billions  of dollars in a transportation infrastructure and therefore, it is not ok and shouldn't be a  part of this culture of, oh,  well, for an environment, I'll  just take now an autonomous vehicle which is an electric  vehicle versus a lytt or uber using fossil fuels and I feel better about it.  That is not a better choice and they shouldn't feel better about it, but when we accept this, that is a slippery slope.  I want to say that I'm glad to  hear you say 

In [96]:
len(rows)

95

In [101]:
with open('trump_comments.csv', mode='w') as f:
    writer = csv.DictWriter(f, rows[0].keys())
    writer.writeheader()
    writer.writerows(rows)

In [99]:
import pandas

ModuleNotFoundError: No module named 'pandas'